<a href="https://colab.research.google.com/github/ThitipongKidnai/thitipong-kidnai/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import moviepy.editor as mp
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import soundfile as sf


DECIBEL_THRESHOLD = 85  # เกณฑ์เสียงอันตราย (dB)
TIME_THRESHOLD = 120  # เวลาสูงสุดที่ยอมให้เสียงดังเกินกำหนด (วินาที)


video_path = "15.mp4"
audio_path = "15.wav"

#ดึงไฟล์เสียงจากวิดีโอ
video = mp.VideoFileClip(video_path)
video.audio.write_audiofile(audio_path)

#โหลดไฟล์เสียงเพื่อวิเคราะห์
audio, sr = librosa.load(audio_path, sr=44100)  # โหลดเสียงที่ sample rate 44.1kHz

#คำนวณระดับเสียงในหน่วย dB
def calculate_decibels(audio_signal):
    rms = np.sqrt(np.mean(audio_signal**2))  # คำนวณค่า RMS ของสัญญาณเสียง
    decibel_level = 20 * np.log10(rms)  # แปลงค่าเป็นเดซิเบล
    return decibel_level

# แบ่งเสียงเป็นช่วงละ 1 วินาทีเพื่อวิเคราะห์
frame_size = sr  # 1 วินาที = จำนวน sample เท่ากับ sample rate
num_frames = len(audio) // frame_size  # คำนวณจำนวนเฟรมที่สามารถแบ่งได้

decibel_values = []
timestamps = []

for i in range(num_frames):
    start = i * frame_size
    end = start + frame_size
    frame_audio = audio[start:end]
    decibel_values.append(calculate_decibels(frame_audio))
    timestamps.append(i)  # Timestamp ในหน่วยวินาที

#แสดงกราฟระดับเสียง
plt.figure(figsize=(10, 5))
plt.plot(timestamps, decibel_values, color='b', label="Decibel Level")
plt.axhline(y=DECIBEL_THRESHOLD, color='r', linestyle='--', label="85 dB Threshold")
plt.xlabel("Time (seconds)")
plt.ylabel("Decibel Level (dB)")
plt.title("Sound Level Over Time")
plt.legend()
plt.grid()
plt.show()

#ตรวจสอบว่ามีเสียงดังเกิน 85 dB เป็นเวลานานกว่า 2 นาทีหรือไม่
exceed_time = sum([1 for db in decibel_values if db > DECIBEL_THRESHOLD])

if exceed_time >= TIME_THRESHOLD:
    print("⚠️ เสียงดังเกิน 85 dB เป็นเวลานานกว่า 2 นาที! ควรหลีกเลี่ยงบริเวณนี้ ⚠️")
else:
    print("✅ เสียงอยู่ในระดับปลอดภัย")

